## Daily Export Final Lambda 

In [ ]:
import json
import praw
import pandas as pd
import boto3
import time


s3 = boto3.client("s3")

def lambda_handler(event,context):
    bucket = "redditapibucket2"

    reddit = praw.Reddit(
    client_id = "Kxke58vtbFOzeTO95IfkLQ",
    client_secret = "SCq6Gw9eoK5G8q4ylhqj46C3OT95sQ",
    user_agent = "mira")


    posts = []
    uc_subreddit = reddit.subreddit("UkrainianConflict")
    for post in uc_subreddit.top(time_filter="day", limit = 150 ): #use "limit=None"
        posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.created])
    posts = pd.DataFrame(posts, columns=["title", "score", "id", "subreddit", "url", "num_comments", "timestamp"])

    data = posts.to_json(orient = "index" , default_handler=str)
    
    timestr = time.strftime("%Y%m%d")
    
    filename= "redditapi" + timestr + ".json"
    
    uploadByteStream = bytes(json.dumps(data).encode("UTF-8"))

    s3.put_object(Bucket = bucket, Key = filename , Body = data)
    
    print("put complete")
